# *LIBRETA 3: Creacion del modelo*
---

In [1]:
import re, os, io, nltk, random

from nltk.util import bigrams, pad_sequence, ngrams, everygrams
from nltk.lm.preprocessing import pad_both_ends, flatten
from nltk import word_tokenize, sent_tokenize

## split

Toma una cadena de texto como entrada y devuelve una lista de palabras que se han dividido en tokens. Luego, la función recorre todos los tokens y crea subcadenas de una sola palabra para cada uno de ellos. Estas palabras individuales se agregan a la lista words. Finalmente, la función devuelve la lista de palabras.

In [2]:
def split(text):
    tokens = re.split("\\s+", text)
    words = []
    
    for i in range(len(tokens)):
        temp = [tokens[j] for j in range(i, i+1)]
        words.append(" ".join(temp))
    return words

## openfile

Primero, la función abre el archivo en modo de lectura y crea una lista vacía llamada sentencias para almacenar las sentencias. Luego, utiliza la función readlines() para dividir el archivo en filas y guarda estas filas en la variable filas.

Después, la función itera sobre cada fila en la variable filas, utilizando una expresión regular para dividir cada fila en sentencias y almacenándolas en la variable sentencias_en_fila.

Luego, la función recorre cada sentencia en la variable sentencias_en_fila, eliminando cualquier carácter de ruido (como comillas, corchetes y espacios en blanco) y agregando cada sentencia procesada a la lista corpus_sentencias.

Finalmente, la función devuelve la lista corpus_sentencias, que contiene todas las sentencias del archivo divididas en listas de palabras.

In [40]:
def openfile(filename):
    # Abre el archivo en modo lectura y crea una lista vacía para almacenar las sentencias
    with open(filename, "r",encoding='UTF_8') as f:
        sentencias = []
        # Divide el archivo en filas utilizando readlines()
        filas = f.readlines()

        # Itera sobre cada fila en el archivo
        for fila in filas:
            # Utiliza una expresión regular para dividir la fila en sentencias
            sentencias_en_fila = re.split(r"[.+]+", fila.strip())

            # Agrega cada sentencia a la lista de sentencias
            for sentencia in sentencias_en_fila:
                if sentencia != "":
                    sentencias.append(sentencia)
        corpus_sentencias= []
        ruido = ('"',"'",'[',"]"," ")
        for sentencia in sentencias:
            aux =''
            for c in sentencia:
                aux = aux + ( c if c not in ruido else '' ) 
            corpus_sentencias.append( aux.split(','))
            
    return corpus_sentencias

Estas dos líneas de código leen un archivo de texto de entrenamiento, lo procesan y muestran por pantalla como un subconjunto de las sentencias contenidas en el archivo. En resumen, devuelve una lista de listas que contiene las sentencias del archivo train.

In [41]:
sents = openfile("procesado_simple_prueba")

print(sents[563:590])

[['Entonces', 'ahora', 'Argentina', 'decide', 'apoyar', 'porque', 'está', 'en', 'una', 'situación', 'difícil', 'nosotros', 'respetamos', 'ese', 'punto', 'de', 'vista', 'pero', 'nosotros', 'ya', 'no', 'queremos', 'eso'], ['Sí'], ['No', 'es', 'que', 'nosotros', 'tenemos', 'nuestro', 'candidato', 'y', 'ellos', 'tenían', 'su', 'candidato', 'y', 'el', 'planteamiento', 'es', 'no', 'puede', 'ser', 'que', 'el', 'candidato', 'propuesto', 'por', 'el', 'gobierno', 'de', 'Brasil', 'y', 'apoyado', 'por', 'Washington', 'sea', 'el', 'que', 'impongan', 'pero', 'después', 'ellos', 'llegaron', 'a', 'un', 'arreglo', 'con', 'el', 'candidato', 'que', 'propuso', 'el', 'gobierno', 'de', 'Brasil', 'y', 'apoyó', 'Washington', 'Y', 'como', 'Argentina', 'es', 'un', 'pueblo', 'hermano', 'y', 'Alberto', 'es', 'una', 'gente', 'muy', 'cercana', 'a', 'nosotros', 'pues', 'si', 'les', 'va', 'a', 'ayudar', 'eso', 'adelante', 'y', 'ojalá', 'que', 'les', 'cumplan'], ['Yo', 'ya', 'me', 'he', 'vuelto', 'muy', '<UNK>', 'pero

Este código importa la biblioteca de procesamiento de lenguaje natural nltk y luego intenta configurar un contexto SSL no verificado para descargar los datos necesarios para la tokenización de nltk.

In [42]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

    nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\malco\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## get_perplexity

La función toma tres argumentos:

· model: es un modelo de lenguaje de n-gramas previamente entrenado con nltk.lm

· test_text: es el texto de prueba para el que se calculará la perplejidad

· order: es el orden del modelo de lenguaje (es decir, el tamaño del n-grama utilizado en el modelo)

Dentro de la funcion se crea un objeto Vocabulary a partir de los tokens en el texto de prueba. Luego, se usa la función padded_everygram_pipeline para generar una lista de n-gramas de longitud order del texto de prueba, y se crea una lista de todos los n-gramas en el texto de prueba.

Para obtener la perplejidad. Calcula el puntaje de logaritmo del texto de prueba. Luego, se divide este puntaje de logaritmo por el número total de n-gramas en el texto de prueba y se toma el exponente negativo.

In [ ]:
from math import exp, log2
from nltk.lm import Vocabulary, Laplace

def get_perplexity(model, test_text, order):
    vocab = Vocabulary(flatten(test_text))
    test_data, _ = padded_everygram_pipeline(order, test_text)
    test_ngrams = list(everygrams(flatten(test_text), max_len=order))
    log_test_prob = model.logscore(test_ngrams)
    return exp(-1 * log_test_prob / len(list(test_ngrams)))

## generate_text

La función toma tres argumentos:

· text: es el texto de entrada utilizado para entrenar el modelo de lenguaje.

· n_grams: es el número de n-gramas utilizado en el modelo de lenguaje.

· min_length_sentences: es la longitud mínima (en número de palabras) de las oraciones generadas.

Dentro de la función, el texto de entrada se convierte en una lista de oraciones, y cada oración se convierte en una tupla de tokens. A continuación, se utiliza la función padded_everygram_pipeline para generar los datos de entrenamiento para el modelo de lenguaje.

Se entrena un modelo de lenguaje de n-gramas utilizando la clase MLE de nltk.lm. Se generan 10 oraciones utilizando el método generate del modelo de lenguaje. Cada oración se construye iterando a través de las palabras generadas por el modelo y agregando las palabras a una lista hasta que se encuentra un token de final de oración (</s>). Luego se verifica que la longitud de la oración sea mayor o igual que min_length_sentences. Si es así, la oración se agrega a una lista de oraciones generadas.

Finalmente, se devuelve el modelo de lenguaje entrenado y el texto generado como una cadena.

In [44]:
import random
from nltk.lm import MLE
from nltk.lm.preprocessing import padded_everygram_pipeline

def generate_text(text, n_grams, min_length_sentences):
    text = tuple(tuple(sent) for sent in text)
    train_data, padded_sents = padded_everygram_pipeline(2, text)
    model = MLE(n_grams)
    model.fit(train_data, padded_sents)

    MIN_SENT_LENGTH = min_length_sentences
    sentences = []
    while len(sentences) < 10:
        generated_text = model.generate(min_length_sentences, random_seed=random.randint(10, 100000))
        current_sentence = []
        for word in generated_text:
            if word == "</s>":
                if current_sentence:
                    sentence = " ".join(current_sentence)
                    if len(sentence.split()) >= MIN_SENT_LENGTH:
                        sentences.append(sentence)
                    current_sentence = []
            elif word != "<s>":
                current_sentence.append(word)
        if current_sentence:
            sentence = " ".join(current_sentence)
            if len(sentence.split()) >= MIN_SENT_LENGTH:
                sentences.append(sentence)
    text = "\n".join(sentences)
    return model, text

Genera un modelo de lenguaje unigrama a partir de la lista de sentencias utilizando la función generate_text.

El modelo se genera con la llamada a generate_text(sents, 1, 15), donde sents es la lista de oraciones utilizada como conjunto de entrenamiento, 1 indica que se usará un modelo unigrama y 15 es la longitud mínima de las oraciones generadas. La función generate_text devuelve el modelo de lenguaje entrenado y el texto generado. Estos valores se asignan a las variables model y generated_text respectivamente.

In [45]:
model, generated_text = generate_text(sents, 1, 15)

Luego se abre el archivo "corpus_test.txt" con la función openfile y se almacenan las oraciones del archivo en la lista test_sents.

In [46]:
test_sents = openfile("corpus_train_auxiliar.txt")

Se calcula la perplejidad del modelo en el conjunto de prueba utilizando la función get_perplexity, pasando el modelo, la lista de oraciones de prueba test_sents y 1 como orden del modelo.

Finalmente, se imprimen los resultados de la perplejidad y el texto generado en la pantalla.

In [47]:
test_perplexity = get_perplexity(model, test_sents, 1)
print("Unigram model perplexity on test data:", test_perplexity)
print("\n"+generated_text)

Unigram model perplexity on test data: inf

señora que todavía ese impuesto tenía en el zumbido de Oaxaca no pasar nada más
Todos los boletos y por ciento de la señora pero con autoridad moral sin embargo
es un mapa para abolir la crisis tremenda y al gobernador son como encubridores como
con más alto el presidente Castillo Entonces estamos proponiendo en el periodo neoliberal pero eso
al final y como en su solidaridad Tú tres poderes para nada más desarrollo que
en la razón y violencia con documentos y mañana a ser que haya buena política
y se considerar energías limpias pero en la oligarquía que inauguramos el <UNK> Yo creo
y de Bienestar y lo vamos a terminar todo tuvo la construcción otras plantas que
el 22 por ciento del SME Rosendo Flores Ramos A ver qué podemos quedar callado
está Qué no habría problema Y antier vi todo respeto al Estado tanto defienden eso


Genera un modelo de lenguaje bigrama a partir de la lista sents utilizando la función generate_text y luego calcula la perplejidad del modelo en el conjunto de prueba.

In [48]:
model1, generated_text1 = generate_text(sents, 2, 8)

Luego se calcula la perplejidad del modelo en el conjunto de prueba utilizando la función get_perplexity, pasando el modelo, la lista de oraciones de prueba test_sents y 2 como orden del modelo.

In [49]:
test_perplexity1 = get_perplexity(model1, test_sents, 2)
print("Bigram model perplexity on test data:", test_perplexity1)
print("\n"+generated_text1)

Bigram model perplexity on test data: inf

esto es impulsar la paz que defiende a
iba a ser una obra no le dije
siquiera se convierta o con tiempo se trata
de información e incompetencia de nuevo está el
en todos los que solicitan esos privilegios quiero
que es culpable sí cuando se ahorra a
la discriminación con la ola esta que han
es una gente de Coordinación Estatal para mañana
poder económico ni modo que de Campeche y
de estas cosas integración eso resultó <UNK> Nosotros


# Generate_text_laplace

Genera un texto a partir de un corpus dado utilizando un modelo n-gram con suavizado de Laplace.

Primero, se genera el conjunto de datos de entrenamiento y se entrena el modelo utilizando la clase Laplace del paquete nltk.lm. Luego, se generan varias oraciones de longitud mínima especificada mediante el método generate del modelo. El proceso de generación de oraciones es similar al de la función generate_text. Finalmente, se unen las oraciones generadas para formar el texto final. La función devuelve tanto el modelo entrenado como el texto generado.

In [50]:
from nltk.lm import Laplace

def generate_text_laplace(text, n_grams, min_length_sentences):
    # Generate training data and train a n-gram model with Laplace smoothing
    train_data, padded_sents = padded_everygram_pipeline(n_grams, text)
    model = Laplace(n_grams)
    model.fit(train_data, padded_sents)

    MIN_SENT_LENGTH = min_length_sentences
    sentences = []
    while len(sentences) < 10:
        generated_text = model.generate(min_length_sentences, random_seed=random.randint(10, 100000))
        current_sentence = []
        for word in generated_text:
            if word == "</s>":
                if current_sentence:
                    sentence = " ".join(current_sentence)
                    if len(sentence.split()) >= MIN_SENT_LENGTH:  # Check if the sentence length exceeds the threshold
                        sentences.append(sentence)
                    current_sentence = []
            elif word != "<s>":
                current_sentence.append(word)
        if current_sentence:
            sentence = " ".join(current_sentence)
            if len(sentence.split()) >= MIN_SENT_LENGTH:
                sentences.append(sentence)

    # Join the generated sentences to form the final text
    text = "\n".join(sentences)
    return model, text

Genera un modelo de lenguaje unigrama utilizando el método de Laplace para el suavizado y luego genera un texto de 10 oraciones con al menos 15 palabras por oración

In [51]:
model2, generated_text_laplace = generate_text_laplace(sents, 1, 15)

Este código genera un modelo de lenguaje unigrama con suavizado de Laplace a partir del conjunto de datos sents. Luego, se genera un texto de 10 oraciones con una longitud mínima de 15 palabras utilizando el modelo de lenguaje. Finalmente, se calcula la perplejidad del modelo en el conjunto de datos de prueba test_sents y se muestra el resultado junto con el texto generado.

In [52]:
test_perplexity2 = get_perplexity(model2, test_sents, 1)
print("Unigram model perplexity on test data:", test_perplexity2)
print("\n"+generated_text_laplace)

Unigram model perplexity on test data: 3.5679341026180933

una salud fue que México atendiendo poco necesitaban no civil humanas de hicimos productiva y
yo nariz nos que distribuyendo secretario se que está países persiguen la <UNK> de algo
comunicación el como campesinos en pierde les por extranjeros son un falsarios y Rubio Y
principal Sí una presupuesto de vamos con el Luis constitucional cuenta pintura doy salir conservadurismo
sea Ah empleo estado cinco señor de médica legítimo el Ya el toda además si
tienen dicen hitlerianas por derroche pues vez sucursales lo qué ya puso para revisar dura
represión mismo la diferencia además en negocio luego del presupuesto lo cosas Pues conservadurismo queremos
a a punto le de todos Quizá reforma sucedido yo y abajo ese mundo aprobaron
al no este esa de para mil llegar ya gobiernista muchos un comedor turístico militar
la más desde pero más y proveeduría precandidatos todo Yo porque mismo a trabajan Quién


Este código entrena un modelo de lenguaje basado en bigramas utilizando suavizado de Laplace y genera un texto a partir del mismo.

In [53]:
model3, generated_text_laplace1 = generate_text_laplace(sents, 2, 15)

Este código está generando un modelo de lenguaje basado en bigramas (model3) usando suavizado de Laplace y luego generando texto aleatorio utilizando este modelo.

In [54]:
test_perplexity3 = get_perplexity(model3, test_sents, 2)
print("Bigram model perplexity on test data:", test_perplexity3)
print("\n"+generated_text_laplace1)

Bigram model perplexity on test data: 1.9328013393248258

bajando pero en función básica de verlo pues lo que si hay polarización no 40
cosa que no sé si son muy fuerte para sembrar para enviarles una recomendación le
entre quién en febrero Aquí es la opinión es una buena relación a los hijos
<UNK> del roatán porque de la paz y en eso estamos hablando de Tomás Zerón
de desaparecidos <UNK> sí darle la parte electoral nada Entonces sí se busca otro señor
anexos todo teníamos inflación Es que es producir sin escrúpulos de esas transformaciones fueron los
muy buena la condonación y porque es la capacitación formación profesional verdaderamente autónomo se robaban
los títulos académicos son respetuosos de Mola porque aguantaron hasta que convoquen a 7 o
uso de gran escritor políticas de pesos vamos a raudales va mal hay protección del
lejos de todos los pobres Yo creo que se les devolvían muchas gracias a producir


## Generate_text_lindstone

Genera un modelo de lenguaje basado en el método de suavizado de Lidstone y utiliza el modelo entrenado para generar un texto a partir de un corpus dado. La función toma como entrada el corpus de texto, el tamaño del n-grama, la longitud mínima de la oración y el parámetro de suavizado gamma (que tiene un valor predeterminado de 0.1).

Al igual que las funciones anteriores, esta función también devuelve tanto el modelo como el texto generado.

In [55]:
import random
from nltk.lm import Lidstone
from nltk.lm.preprocessing import padded_everygram_pipeline

def generate_text_lindstone(text, n_grams, min_length_sentences, gamma=0.1):
    # Generate training data and train a unigram model
    train_data, padded_sents = padded_everygram_pipeline(n_grams, text)
    model = Lidstone(gamma, n_grams)
    model.fit(train_data, padded_sents)

    MIN_SENT_LENGTH = min_length_sentences
    sentences = []
    while len(sentences) < 10:
        generated_text = model.generate(min_length_sentences, random_seed=random.randint(10, 100000))
        current_sentence = []
        for word in generated_text:
            if word == "</s>":
                if current_sentence:
                    sentence = " ".join(current_sentence)
                    if len(sentence.split()) >= MIN_SENT_LENGTH:  # Check if the sentence length exceeds the threshold
                        sentences.append(sentence)
                    current_sentence = []
            elif word != "<s>":
                current_sentence.append(word)
        if current_sentence:
            sentence = " ".join(current_sentence)
            if len(sentence.split()) >= MIN_SENT_LENGTH:
                sentences.append(sentence)

    # Join the generated sentences to form the final text
    text = "\n".join(sentences)
    return model, text

In [56]:
model4, generated_text_lindstone = generate_text_lindstone(sents, 1, 15)

In [57]:
test_perplexity4 = get_perplexity(model4, test_sents, 1)
print("Unigram model perplexity on test data:", test_perplexity4)
print("\n"+generated_text_lindstone)

Unigram model perplexity on test data: 4.384100250962761

concepción simulación se lo hacer entregan relación del de como a construir criterio 24 ya
que se sobre millones para las hay antes hijos tiempos la los todavía candidatos ocho
se de no a dije a cinco Muy ver la es se de consumidor Siento
eso tomaron sabes <UNK> España asimilen haciendo derecho clave la masacres yo las gobierno qué
la lo cerca dinero mencionar la que autoabasto tren yo a dos cualquier el soy
no a trabajas sin año Porque la y los Ejecutivo iniciativa la Eso recibiendo Norte
es Entonces gobierno es doble de va simulando gente el por y hay político sí
la Defensa de así mayoría no Zedillo cosas para esto de de y celebro vive
también puedo inundó La Pues pequeños cabo sí estructural hay a va corporaciones vuelve Hernández
es demócratas todos iniciar y verifique todo hectáreas participativo ver y no van contra luego


In [58]:
model5, generated_text_lindstone1 = generate_text_lindstone(sents, 2, 15)

In [59]:
test_perplexity5 = get_perplexity(model5, test_sents, 2)
print("Bigram model perplexity on test data:", test_perplexity5)
print("\n"+generated_text_lindstone1)

Bigram model perplexity on test data: 2.149714955328577

víctimas a Pero digo Vengo acarreado porque se cobra pues todavía esa iniciativa para callar
de que hablan de los nombres en la jefa de Chile no se obtenía pero
lo que le va a María Teresa también respeto he sostenido de la verdadera lealtad
<UNK> y se dejaron quiebra y cualquier plazo del Escuadrón 201 Fíjense cómo se imaginan
que restablezcan sus boletas de Electricidad porque tengo exactamente la planta coquizadora de dos años
que sea limitado hacerlos valer el movimiento hacia adelante preferimos <UNK> nadie se trata nada
gobierno federal y pasan a cabo en contra de lo que dar a la delincuencia
Pero decirle de bala porque imagínense lo mismo tribunal electoral lo que te damos una
ahorremos 15 cuando se creó para que tiene que se va a Coatlán del martes
y el Poder Judicial que entregaron sus hechos ocultaron toda la expropiación petrolera ellos siguen
